In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from pickle import dump
import gc
import time

In [2]:
start_time = time.time()

In [3]:
#pairs = pd.read_pickle('../input/fourpoints-location-matching/train_pairs.pkl').sample(n=10000, random_state=1).select_dtypes(exclude=["category","object_"])
pairs = pd.read_pickle('../input/fourpoints-location-matching/train_pairs.pkl').select_dtypes(exclude=["category","object_"])
pairs.drop(['country_2_count','latitude_2','longitude_2'],axis = 1,inplace = True)
gc.collect()

0

In [4]:
train_y = pairs['match'].astype('int8')

In [5]:
pairs[['categories_1_count', 'categories_2_count','city_1_count','city_2_count','state_1_count','state_2_count']] = pairs[['categories_1_count', 'categories_2_count','city_1_count','city_2_count','state_1_count','state_2_count']].fillna(value=0)
pairs['categories_1_count_diff'] = pairs['categories_1_count']-pairs['categories_2_count']
pairs['city_1_count_diff'] = pairs['city_1_count']-pairs['city_2_count']
pairs['state_1_count_diff'] = pairs['state_1_count']-pairs['state_2_count']
gc.collect()

13

In [6]:
#pairs.info()

In [7]:
#pairs_words = pd.read_pickle('../input/fourpoints-location-matching/train_pairs_words.pkl').sample(n=10000, random_state=1)
pairs_words = pd.read_pickle('../input/fourpoints-location-matching/train_pairs_words.pkl')

gc.collect()
gc.collect()

0

In [8]:
pairs = pd.concat([pairs,pairs_words], axis = 1)
del pairs_words
gc.collect()

23

In [9]:
pairs.select_dtypes(exclude=["category","object_"]).columns.tolist()

['address_1_missing',
 'address_2_missing',
 'categories_1_count',
 'categories_1_count_diff',
 'categories_2_count',
 'city_1_count',
 'city_1_count_diff',
 'city_1_missing',
 'city_2_count',
 'city_2_missing',
 'country_1_count',
 'country_1_count_diff',
 'distance',
 'latitude_1',
 'latitude_1_count',
 'latitude_2_count',
 'latitude_2_count_diff',
 'longitude_1',
 'longitude_1_count',
 'longitude_2_count',
 'longitude_2_count_diff',
 'match',
 'phone_1_missing',
 'phone_2_missing',
 'state_1_count',
 'state_1_count_diff',
 'state_2_count',
 'url_1_missing',
 'url_2_missing',
 'zip_1_missing',
 'zip_2_missing',
 'name_1_fuzzy',
 'name_1_fuzzy_partial',
 'address_1_fuzzy',
 'categories_1_fuzzy',
 'categories_1_fuzzy_partial',
 'url_1_fuzzy']

In [10]:
cols = ['address_1_missing',
 'address_2_missing',
 'categories_1_count',
 'categories_1_count_diff',
 'categories_2_count',
 'city_1_count',
 'city_1_count_diff',
 'city_1_missing',
 'city_2_count',
 'city_2_missing',
 'country_1_count',
 'country_1_count_diff',
 'distance',
 'latitude_1',
 'latitude_1_count',
 'latitude_2_count',
 'latitude_2_count_diff',
 'longitude_1',
 'longitude_1_count',
 'longitude_2_count',
 'longitude_2_count_diff',
 'phone_1_missing',
 'phone_2_missing',
 'state_1_count',
 'state_1_count_diff',
 'state_2_count',
 'url_1_missing',
 'url_2_missing',
 'zip_1_missing',
 'zip_2_missing',
 'name_1_fuzzy',
 'name_1_fuzzy_partial',
 'address_1_fuzzy',
 'categories_1_fuzzy',
 'categories_1_fuzzy_partial',
 'url_1_fuzzy']
ids = ['id_1','id_2']

In [11]:
#train_id = pairs[ids]
train_X = pairs[cols]
#train_X['country_same'] = np.where(train_X['country_1'].astype(object) == train_X['country_2'].astype(object),1,0)
#del train_X['country_1'] 
#del train_X['country_2']
del pairs
gc.collect()

69

In [12]:
gc.collect()
gc.collect()

0

In [13]:
#col_scaler = ['latitude_1','longitude_1','latitude_2','longitude_2']
#X_transform = train_X.copy()
scaler = RobustScaler().fit(train_X.sample(n=5000000, random_state=1))
dump(scaler, open('scaler.pkl', 'wb')) # save the scaler
train_X = scaler.transform(train_X)
gc.collect()

23

In [14]:
#train_X

In [ ]:
#train_y = pairs['match'].astype('int')
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
              importance_type='gain',
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=2, n_estimators=400, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.9, tree_method='exact', validate_parameters=1,
              verbosity=2)
xgb.fit(train_X, train_y)
gc.collect()
xgb.save_model("model.json")

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))

# download model
<a href="./scaler.pkl"> scaler </a>

<a href="./model.json"> xgboost model </a>

In [ ]:
#pairs.dtypes.apply(lambda x: x.name).to_dict()

{'id_1': 'category',
 'name_1': 'object',
 'latitude_1': 'float16',
 'longitude_1': 'float16',
 'address_1': 'object',
 'city_1': 'category',
 'state_1': 'category',
 'zip_1': 'category',
 'country_1': 'category',
 'url_1': 'int8',
 'phone_1': 'int8',
 'categories_1': 'object',
 'id_2': 'category',
 'name_2': 'object',
 'latitude_2': 'float16',
 'longitude_2': 'float16',
 'address_2': 'object',
 'city_2': 'category',
 'state_2': 'category',
 'zip_2': 'category',
 'country_2': 'category',
 'url_2': 'int8',
 'phone_2': 'int8',
 'categories_2': 'object',
 'match': 'float16',
 'distance': 'float16',
 'name_1_fuzzy': 'float64',
 'address_1_fuzzy': 'float64',
 'categories_1_fuzzy': 'float64'}

['address_1_missing',
 'address_2_missing',
 'categories_1_count',
 'categories_1_count_diff',
 'categories_2_count',
 'city_1_count',
 'city_1_count_diff',
 'city_1_missing',
 'city_2_count',
 'city_2_missing',
 'country_1_count',
 'country_1_count_diff',
 'distance',
 'latitude_1',
 'latitude_1_count',
 'latitude_2_count',
 'latitude_2_count_diff',
 'longitude_1',
 'longitude_1_count',
 'longitude_2_count',
 'longitude_2_count_diff',
 'match',
 'phone_1_missing',
 'phone_2_missing',
 'state_1_count',
 'state_1_count_diff',
 'state_2_count',
 'url_1_missing',
 'url_2_missing',
 'zip_1_missing',
 'zip_2_missing']